In [1]:
#| default_exp gsc_db

In [2]:
#| export
from pymongo import MongoClient, ASCENDING, DESCENDING
from typing import List,Dict
from datetime import datetime

In [15]:
#| export
class SearchConsoleDB:
    def __init__(self, connection_uri: str = "mongodb://localhost:27017/"):
        """Initialize MongoDB connection"""
        self.client = MongoClient(connection_uri)
        self.db = self.client.search_console
        
        # Create indexes for better query performance
        self.db.analytics.create_index([
            ("site_url", ASCENDING),
            ("date", ASCENDING),
            ("dimension_type", ASCENDING)
        ])

    def store_search_analytics(self, site_url: str, data: List[Dict], dimension_type: str):
        """Store search analytics data with duplicate prevention"""
        timestamp = datetime.now()
        today = timestamp.strftime('%Y-%m-%d')
        
        # Check for existing data today
        existing = self.db.analytics.find_one({
            'site_url': site_url,
            'date': today,
            'dimension_type': dimension_type
        })
        
        if existing:
            # Update existing data instead of inserting new
            self.db.analytics.delete_many({
                'site_url': site_url,
                'date': today,
                'dimension_type': dimension_type
            })
        
        documents = []
        for item in data:
            doc = {
                'site_url': site_url,
                'timestamp': timestamp,
                'date': today,
                'dimension_type': dimension_type,
                'metrics': {
                    'clicks': item['clicks'],
                    'impressions': item['impressions'],
                    'ctr': item['ctr'],
                    'position': item['position']
                },
                'dimensions': {}
            }
            
            # Add dimension values (query, page, country, device, etc.)
            for key in item.get('keys', []):
                doc['dimensions'][dimension_type] = key
            
            documents.append(doc)
        
        if documents:
            self.db.analytics.insert_many(documents)
    def get_analytics(self, site_url: str, 
                    start_date: str,
                    end_date: str,
                    dimension_type: str = None,
                    metrics_filter: Dict = None):
        """Get analytics data with flexible filtering"""
        query = {
            'site_url': site_url,
            'date': {
                '$gte': start_date,
                '$lte': end_date
            }
        }
        
        if dimension_type:
            query['dimension_type'] = dimension_type
        
        if metrics_filter:
            for metric, value in metrics_filter.items():
                query[f'metrics.{metric}'] = value
        
        return list(self.db.analytics.find(query))
    def get_top_queries(self, site_url: str, 
                    start_date: str,
                    end_date: str,
                    metric: str = 'clicks',
                    limit: int = 10):
        """Get top performing queries"""
        pipeline = [
            {
                '$match': {
                    'site_url': site_url,
                    'date': {
                        '$gte': start_date,
                        '$lte': end_date
                    },
                    'dimension_type': 'query'
                }
            },
            {
                '$group': {
                    '_id': '$dimensions.query',
                    'total_clicks': {'$sum': '$metrics.clicks'},
                    'total_impressions': {'$sum': '$metrics.impressions'},
                    'avg_position': {'$avg': '$metrics.position'},
                    'avg_ctr': {'$avg': '$metrics.ctr'}
                }
            },
            {
                '$sort': {f'total_{metric}': -1}
            },
            {
                '$limit': limit
            }
        ]
        
        return list(self.db.analytics.aggregate(pipeline))
    
    def get_trends(self, site_url: str,
                start_date: str,
                end_date: str,
                dimension: str = None):
        """Get trends over time"""
        pipeline = [
            {
                '$match': {
                    'site_url': site_url,
                    'date': {
                        '$gte': start_date,
                        '$lte': end_date
                    }
                }
            },
            {
                '$group': {
                    '_id': {
                        'date': '$date',
                        'dimension': f'$dimensions.{dimension}' if dimension else None
                    },
                    'clicks': {'$sum': '$metrics.clicks'},
                    'impressions': {'$sum': '$metrics.impressions'},
                    'avg_position': {'$avg': '$metrics.position'},
                    'avg_ctr': {'$avg': '$metrics.ctr'}
                }
            },
            {
                '$sort': {'_id.date': 1}
            }
        ]
        
        return list(self.db.analytics.aggregate(pipeline))

    def get_page_analytics(self, site_url: str,
                        page_path: str,  # e.g., "/blog/article-1" or full URL
                        start_date: str,
                        end_date: str,
                        include_queries: bool = True):
        """Get analytics for a specific page/article"""
        
        # Handle both full URLs and paths
        if page_path.startswith('http'):
            page_path = page_path.split(site_url.replace('sc-domain:', ''))[-1]
        
        pipeline = [
            {
                '$match': {
                    'site_url': site_url,
                    'date': {
                        '$gte': start_date,
                        '$lte': end_date
                    },
                    'dimensions.page': {'$regex': f'.*{page_path}$'}
                }
            },
            {
                '$group': {
                    '_id': {
                        'date': '$date',
                        'page': '$dimensions.page',
                        'query': '$dimensions.query' if include_queries else None
                    },
                    'clicks': {'$sum': '$metrics.clicks'},
                    'impressions': {'$sum': '$metrics.impressions'},
                    'avg_position': {'$avg': '$metrics.position'},
                    'avg_ctr': {'$avg': '$metrics.ctr'}
                }
            },
            {
                '$sort': {
                    '_id.date': 1,
                    'clicks': -1
                }
            }
        ]
        
        results = list(self.db.analytics.aggregate(pipeline))
        
        # Format the results
        return {
            'page_path': page_path,
            'total_clicks': sum(r['clicks'] for r in results),
            'total_impressions': sum(r['impressions'] for r in results),
            'average_position': sum(r['avg_position'] for r in results) / len(results) if results else 0,
            'average_ctr': sum(r['avg_ctr'] for r in results) / len(results) if results else 0,
            'daily_metrics': results,
            'top_queries': [r for r in results if r['_id']['query']] if include_queries else []
        }
